In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna

from ium_long_stay_patterns.src.helpers.create_numerical_dataset import create_numerical_dataset, merge_with_stats
from ium_long_stay_patterns.config import ProcessedCSV, SAVED_MODELS_DIR
from ium_long_stay_patterns.src.helpers.data_loaders import prepare_and_create_loaders
from ium_long_stay_patterns.modeling.train import Trainer
from models.binary import BinaryClassifier

import warnings
warnings.filterwarnings('ignore')

/home/matimat/.cache/pypoetry/virtualenvs/ium-long-stay-patterns-kVcCAgnW-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-13 20:16:51.810 | INFO     | ium_long_stay_patterns.config:<module>:14 - PROJ_ROOT path is: /home/matimat/IUM/ium-long-stay-patterns


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
df_numeric = create_numerical_dataset(ProcessedCSV.LISTINGS.path, strategy=True)
df_final = merge_with_stats(df_numeric, with_ids=True)

X = df_final.drop(columns=['target', 'id', 'host_id', 'listing_id'])
y = df_final['target']

print(f"Dataset shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")

Dataset shape: (1368, 18)
Target distribution:
target
0    994
1    374
Name: count, dtype: int64


In [4]:
def objective(trial):
    """
    Objective function for Optuna to optimize.
    Returns validation AUC score.
    """
    n_layers = trial.suggest_int('n_layers', 1, 3)

    hidden_layers = []
    for i in range(n_layers):
        hidden_size = trial.suggest_int(f'n_units_l{i}', 16, 128, step=16)
        hidden_layers.append(hidden_size)

    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)

    train_loader, val_loader, _, _ = prepare_and_create_loaders(
        X, y, batch_size=batch_size, random_state=42, save_test_data=False, verbose=False
    )

    data_iter = iter(train_loader)
    sample_batch, _ = next(data_iter)
    input_dim = sample_batch.shape[1]

    model = BinaryClassifier(
        input_dim=input_dim,
        hidden_layers=hidden_layers,
        dropout_rate=dropout_rate
    ).to(device)

    optimizer = optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    criterion = nn.BCELoss()

    trainer = Trainer(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        epochs=10,
        device=device,
        seed=42
    )

    for epoch in range(1, 51):
        trainer.model.train()
        epoch_loss = 0.0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 10 == 0:
            metrics = trainer._validate(val_loader)

            trial.report(metrics['auc'], epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    final_metrics = trainer._validate(val_loader)

    return final_metrics['auc']

In [5]:
study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    study_name='binary_classifier_tuning'
)

print("Starting hyperparameter optimization...")
study.optimize(objective, n_trials=50, timeout=3600)

print("\nOptimization complete!")

[I 2026-01-13 20:16:53,525] A new study created in memory with name: binary_classifier_tuning


Starting hyperparameter optimization...


[I 2026-01-13 20:16:59,713] Trial 0 finished with value: 0.9180293501048218 and parameters: {'n_layers': 3, 'n_units_l0': 96, 'n_units_l1': 112, 'n_units_l2': 80, 'dropout_rate': 0.0, 'learning_rate': 0.004325844245358131, 'batch_size': 64, 'weight_decay': 7.7281355662327e-05}. Best is trial 0 with value: 0.9180293501048218.
[I 2026-01-13 20:17:01,803] Trial 1 finished with value: 0.8163522012578617 and parameters: {'n_layers': 3, 'n_units_l0': 80, 'n_units_l1': 64, 'n_units_l2': 16, 'dropout_rate': 0.0, 'learning_rate': 0.0003741309800128861, 'batch_size': 128, 'weight_decay': 7.2795713237901115e-06}. Best is trial 0 with value: 0.9180293501048218.
[I 2026-01-13 20:17:05,248] Trial 2 finished with value: 0.810796645702306 and parameters: {'n_layers': 3, 'n_units_l0': 48, 'n_units_l1': 64, 'n_units_l2': 112, 'dropout_rate': 0.5, 'learning_rate': 0.0004429722381496332, 'batch_size': 64, 'weight_decay': 3.1114854344513767e-06}. Best is trial 0 with value: 0.9180293501048218.
[I 2026-01-1


Optimization complete!


In [6]:
# Best trial
print("Best trial:")
trial = study.best_trial

print(f"  Value (AUC): {trial.value:.4f}")
print("\n  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value (AUC): 0.9536

  Params: 
    n_layers: 3
    n_units_l0: 80
    n_units_l1: 128
    n_units_l2: 64
    dropout_rate: 0.30000000000000004
    learning_rate: 0.008375350811026756
    batch_size: 64
    weight_decay: 0.0007186556345221058


## Top 10 best trials

In [7]:
df = study.trials_dataframe()

df_sorted = df.sort_values("value", ascending=False)

top10 = df_sorted.head(10)

cols = ["number", "value"] + [c for c in df_sorted.columns if c.startswith("params_")]

display(top10[cols])

,number,value,params_batch_size,params_dropout_rate,params_learning_rate,params_n_layers,params_n_units_l0,params_n_units_l1,params_n_units_l2,params_weight_decay
11,11,0.953564,64,0.3,0.008375,3,80,128.0,64.0,0.000719
12,12,0.952830,64,0.3,0.009775,3,80,16.0,48.0,0.000874
43,43,0.950105,64,0.3,0.009365,3,80,112.0,48.0,0.000479
21,21,0.945702,64,0.3,0.008662,3,80,128.0,64.0,0.000883
27,27,0.945073,64,0.2,0.004494,2,80,48.0,NaN,0.000639
10,10,0.942662,64,0.3,0.009013,3,80,128.0,64.0,0.000791
23,23,0.942034,64,0.3,0.005982,3,80,96.0,96.0,0.000430
35,35,0.936373,64,0.3,0.009931,1,112,NaN,NaN,0.000611
22,22,0.930818,64,0.4,0.009488,3,64,128.0,64.0,0.000924
42,42,0.925472,64,0.3,0.008098,3,96,128.0,64.0,0.000821


Params saved in *config.py*

# Train best model - in *classification_numeric_data.ipynb*